In [ ]:
def split_string_by_newline(s: str) -> list[str]:
    """
    将字符串从第1000个字符开始，按'\n'、'\\n'或字符串结尾进行划分，返回字符串数组
    
    参数:
        s: 要划分的原始字符串
        
    返回:
        list[str]: 划分后的字符串列表
    """
    result = []
    start = 0
    chunk_size = 1000  # 从第1000个字符开始寻找分割点
    
    while start < len(s):
        # 计算当前 chunk 的结束位置（从start开始的第1000个字符）
        end_candidate = start + chunk_size
        
        # 如果剩余字符不足1000，直接取到结尾
        if end_candidate >= len(s):
            result.append(s[start:])
            break
        
        # 从第1000个字符位置开始寻找分割点
        # 搜索范围：end_candidate 到字符串结尾（避免漏检）
        search_start = end_candidate
        split_pos = -1
        
        # 优先寻找'\n'（实际换行符）
        split_pos = s.find('\n', search_start)
        
        # 如果没找到'\n'，寻找'\\n'（转义的换行符）
        if split_pos == -1:
            split_pos = s.find('\\n', search_start)
            # 找到'\\n'时，分割位置需要包含两个字符
            if split_pos != -1:
                split_pos += 2  # 指向'\\n'之后的位置
        
        # 如果两种换行符都没找到，分割到字符串结尾
        if split_pos == -1:
            split_pos = len(s)
        
        # 将当前 chunk 加入结果列表
        result.append(s[start:split_pos])
        
        # 更新下一个 chunk 的起始位置
        start = split_pos
    
    return result

In [ ]:
import os
import json
from tqdm import tqdm
# 文件路径配置
input_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion.json"
output_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split.json"

# 检查输入文件是否存在
if not os.path.exists(input_path):
    raise FileNotFoundError(f"输入文件未找到：{input_path}")

# 读取原始数据集
with open(input_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

print(f"✅ 成功加载数据集，共 {len(dataset)} 条记录")

# 处理每条记录
processed_dataset = []
for item in tqdm(dataset, desc="处理进度"):
    # 复制原始字段（保留completion）
    processed_item = {
        "completion": item.get("completion", "")
    }
    
    # 处理prefix划分
    prefix = item.get("prefix", "")
    prefix_splits = split_string_by_newline(prefix)
    for i, split in enumerate(prefix_splits, 1):
        processed_item[f"prefix_split_{i}"] = split
    
    # 处理suffix划分
    suffix = item.get("suffix", "")
    suffix_splits = split_string_by_newline(suffix)
    for i, split in enumerate(suffix_splits, 1):
        processed_item[f"suffix_split_{i}"] = split
    
    processed_dataset.append(processed_item)

# 保存处理后的数据集
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(processed_dataset, f, ensure_ascii=False, indent=2)

# 验证结果
print(f"\n✅ 数据集处理完成，已保存至：{output_path}")

# 打印第一条记录的结构，确认划分结果
if processed_dataset:
    first_item = processed_dataset[0]
    print("\n第一条记录的划分结果（字段列表）：")
    print(list(first_item.keys()))
    
    # 统计prefix和suffix的划分数量
    prefix_split_count = sum(1 for key in first_item if key.startswith("prefix_split_"))
    suffix_split_count = sum(1 for key in first_item if key.startswith("suffix_split_"))
    print(f"\n第一条记录中：")
    print(f"  prefix被划分为 {prefix_split_count} 部分")
    print(f"  suffix被划分为 {suffix_split_count} 部分")

In [ ]:
def reverse_trunc_string(s: str) -> str:
    """
    从字符串末端反向计数1000个字符，向前寻找第一个'\n'或'\\n'，
    返回从分割点到末端的子字符串，并去除开头的换行符和空格
    """
    if len(s) <= 1000:
        truncated = s
    else:
        start_pos = len(s) - 1000
        # 优先寻找'\n'
        split_pos = s.rfind('\n', 0, start_pos + 1)
        
        # 寻找'\\n'
        if split_pos == -1:
            split_pos = s.rfind('\\n', 0, start_pos + 1)
            if split_pos != -1:
                split_pos += 2  # 从转义义换行符后开始取
        
        # 未找到换行符则从start_pos开始取
        if split_pos == -1:
            split_pos = start_pos
        
        truncated = s[split_pos:]
    
    # 去除除开头的换行符(\n)、转义换行符(\\n)和空格
    # 先处理转义换行符
    while truncated.startswith(('\\n', '\n', ' ')):
        if truncated.startswith('\\n'):
            truncated = truncated[2:]  # 移除\\n
        elif truncated.startswith('\n'):
            truncated = truncated[1:]  # 移除\n
        elif truncated.startswith(' '):
            truncated = truncated.lstrip(' ')  # 移除空格
    
    return truncated

In [ ]:
# 文件路径配置
input_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion.json"
output_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_trunc.json"

# 检查输入文件是否存在
if not os.path.exists(input_path):
    raise FileNotFoundError(f"输入文件未找到：{input_path}")

# 读取原始数据集
with open(input_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

print(f"✅ 成功加载数据集，共 {len(dataset)} 条记录")

# 处理每条记录
processed_dataset = []
for item in tqdm(dataset, desc="处理进度"):
    # 对prefix和suffix进行截断处理
    processed_item = {
        "prefix_trunc_1000": reverse_trunc_string(item.get("prefix", "")),
        "suffix_trunc_1000": reverse_trunc_string(item.get("suffix", "")),
        "completion": item.get("completion", "")  # 保持completion不变
    }
    processed_dataset.append(processed_item)

# 保存处理后的数据集
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(processed_dataset, f, ensure_ascii=False, indent=2)

# 验证结果
print(f"\n✅ 数据集处理完成，已保存至：{output_path}")

# 打印第一条记录的截断前后长度对比
if processed_dataset and dataset:
    first_original = dataset[0]
    first_processed = processed_dataset[0]
    
    print("\n第一条记录截断前后长度对比：")
    print(f"  原始prefix长度：{len(first_original.get('prefix', ''))} → 截断后：{len(first_processed['prefix_trunc_1000'])}")
    print(f"  原始suffix长度：{len(first_original.get('suffix', ''))} → 截断后：{len(first_processed['suffix_trunc_1000'])}")
    print(f"  completion长度：{len(first_processed['completion'])}（未截断）")

In [1]:
import json
import os
from tqdm import tqdm

def clean_code_blocks() -> None:
    """
    处理数据集中所有对象，对prefix_split_*和suffix_split_*字段检查并移除
    前缀("```"或"```python")和后缀("```")，保持与annotate函数一致的遍历风格
    """
    # 文件路径配置
    input_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp.json"
    output_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_cleaned.json"
    
    # 检查输入文件是否存在
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"输入文件未找到：{input_path}")
    
    # 读取数据集（列表中的每个元素是字典）
    with open(input_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    # 验证数据集格式
    if not isinstance(dataset, list):
        raise TypeError("数据集应为列表格式，每个元素应为字典")
    
    print(f"✅ 成功加载数据集，共 {len(dataset)} 条记录")
    
    # 处理所有对象
    processed_dataset = []
    for item in tqdm(dataset, desc="清理代码块标记"):
        # 复制原始对象（保留所有字段）
        cleaned_item = {}
        
        # 处理所有prefix_split字段
        prefix_fields = [key for key in item if key.startswith("prefix_split_")]
        for field in prefix_fields:
            content = item[field]
            # 检查并移除前缀
            if content.startswith("```python"):
                content = content[len("```python"):]
            elif content.startswith("```"):
                content = content[len("```"):]
            # 检查并移除后缀
            if content.endswith("```"):
                content = content[:-len("```")]
            # 去除首尾空白
            cleaned_item[field] = content
        
        # 处理所有suffix_split字段
        suffix_fields = [key for key in item if key.startswith("suffix_split_")]
        for field in suffix_fields:
            content = item[field]
            # 检查并移除前缀
            if content.startswith("```python"):
                content = content[len("```python"):]
            elif content.startswith("```"):
                content = content[len("```"):]
            # 检查并移除后缀
            if content.endswith("```"):
                content = content[:-len("```")]
            # 去除首尾空白
            cleaned_item[field] = content
        
        # 保留其他字段（如completion）
        other_fields = [key for key in item 
                       if not key.startswith("prefix_split_") 
                       and not key.startswith("suffix_split_")]
        for field in other_fields:
            cleaned_item[field] = item[field]
        
        processed_dataset.append(cleaned_item)
    
    # 保存清理后的数据集
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 清理完成！结果保存至：")
    print(f"   {output_path}")
    
    # 验证清理结果
    if processed_dataset:
        first_item = processed_dataset[0]
        # 查找示例字段
        sample_prefix = next((key for key in first_item if key.startswith("prefix_split_")), None)
        sample_suffix = next((key for key in first_item if key.startswith("suffix_split_")), None)
        
        if sample_prefix:
            print(f"\n{sample_prefix} 清理示例（前30字符）：")
            print(f"{first_item[sample_prefix][:30]}...")
        if sample_suffix:
            print(f"\n{sample_suffix} 清理示例（前30字符）：")
            print(f"{first_item[sample_suffix][:30]}...")


# 执行清理
if __name__ == "__main__":
    try:
        clean_code_blocks()
    except Exception as e:
        print(f"处理过程中发生错误：{str(e)}")


✅ 成功加载数据集，共 272 条记录


清理代码块标记: 100%|██████████| 272/272 [00:00<00:00, 200783.30it/s]


✅ 清理完成！结果保存至：
   C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_cleaned.json

prefix_split_1 清理示例（前30字符）：
import asyncio
import websocke...

suffix_split_1 清理示例（前30字符）：
next_token = <LibFunc->(use ge...
